# Машинное обучение, DS-поток
## Задание 1.13


**Правила:**

* Выполненную работу нужно отправить телеграм-боту `@miptstats_ad21_bot`.
* Дедлайны см. в боте. После дедлайна работы не принимаются кроме случаев наличия уважительной причины.
* Прислать нужно ноутбук в формате `ipynb`.
* Решения, размещенные на каких-либо интернет-ресурсах не принимаются. Публикация решения может быть приравнена к предоставлении возможности списать.
* Для выполнения задания используйте этот ноутбук в качестве основы, ничего не удаляя из него.

---

## Генерация текстов с трансформеров.

**За задание можно получить 6 баллов.** 

В данном задании вы будете вновь генерировать тексты на основании высказываний [Ницше](https://ru.wikipedia.org/wiki/Ницше,_Фридрих). 
Ваша задача использовать трансформеры для генерации текстов.

Попробуем улучшить результат по сравнению с реккурентными сетями.

В задании можно использовать как свои, так и предобученные сети


In [1]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 30.8 MB/s 
     |████████████████████████████████| 880 kB 61.5 MB/s 
     |████████████████████████████████| 84 kB 3.8 MB/s 
     |████████████████████████████████| 596 kB 40.2 MB/s 
     |████████████████████████████████| 6.6 MB 38.4 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=be142ee3b7e3edd09720b258f31fca50791112beea3ca0d8c388ba7ed6cf104c
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import io
import os
import sys
import random
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from transformers import LineByLineTextDataset,TextDataset,\
 DataCollatorForLanguageModeling,Trainer, TrainingArguments

from nltk.tokenize import TweetTokenizer

from sklearn.model_selection import train_test_split

import time
from IPython.display import clear_output

from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import logging

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

RANDOM_SEED = 17

Скачиваем данные. Корпус состоит из высказываний Ницше, разделеных переносом строки.

In [3]:
!wget 'https://docs.google.com/uc?export=download&id=1BTyKYpAQ0yLBqHVMQ_12BsLJVyixgp9a' -O nietzsche.txt

--2022-05-12 08:24:21--  https://docs.google.com/uc?export=download&id=1BTyKYpAQ0yLBqHVMQ_12BsLJVyixgp9a
Resolving docs.google.com (docs.google.com)... 142.251.12.102, 142.251.12.113, 142.251.12.100, ...
Connecting to docs.google.com (docs.google.com)|142.251.12.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/5n0bd4qjdf64bulpb4hl7hbtf58cnv1p/1652343825000/14359032242157329066/*/1BTyKYpAQ0yLBqHVMQ_12BsLJVyixgp9a?e=download [following]
--2022-05-12 08:24:23--  https://doc-14-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/5n0bd4qjdf64bulpb4hl7hbtf58cnv1p/1652343825000/14359032242157329066/*/1BTyKYpAQ0yLBqHVMQ_12BsLJVyixgp9a?e=download
Resolving doc-14-4c-docs.googleusercontent.com (doc-14-4c-docs.googleusercontent.com)... 142.251.12.132, 2404:6800:4003:c11::84
Connecting to doc-14-4c-docs.googleusercontent.com (doc-14-4c-docs.goog

In [4]:
!head nietzsche.txt

PREFACE


SUPPOSING that Truth is a woman--what then? Is there not ground
for suspecting that all philosophers, in so far as they have been
dogmatists, have failed to understand women--that the terrible
seriousness and clumsy importunity with which they have usually paid
their addresses to Truth, have been unskilled and unseemly methods for
winning a woman? Certainly she has never allowed herself to be won; and
at present every kind of dogma stands with sad and discouraged mien--IF,


In [5]:
with io.open('./nietzsche.txt', encoding='utf-8') as f:
    text = f.read().lower()
data = text.split('\n')

In [6]:
max_len=0
all_len=[]
for line in data:
    max_len=max(max_len,len(line))
    all_len.append(len(line))
max_len, np.median(all_len),len(data)

(72, 68.0, 9935)

In [7]:
tokenizer = TweetTokenizer()
tokens = list(set(tokenizer.tokenize(text)))
print('total tokens:', len(tokens))

total tokens: 11393


In [8]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokens = list(set(tokenizer.tokenize(text)))
print('total tokens:', len(tokens))

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (143018 > 1024). Running this sequence through the model will result in indexing errors


total tokens: 10680


Предлагаю прочитать вот данную [статью](https://huggingface.co/docs/transformers/tokenizer_summary). В ней говорится, что в GPT-2 используется BPE

Обосную почему я необходимо использовать именно gpt. Еще на самом деле у нас есть bert, но на офиицвльном сайте сказано "BERT was trained with the masked language modeling (MLM) and next sentence prediction (NSP) objectives. It is efficient at predicting masked tokens and at NLU in general, but is not optimal for text generation." Так что для генерации текста лучше всего подойдет gpt. 

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelWithLMHead.from_pretrained("gpt2").to(device)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:911: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

In [10]:
def generate_texts(model, prefix, num_texts=5, max_length=100):
    '''
    Функция для генерации текстов с помощью GPT 

    Параметры.
    1) model — модель, с помощью которой осуществляется генерация, 
    2) prefix — начало текста,
    3) num_texts — количество текстов, которые требуется сгенерировать,
    4) max_length — максимальное количество добавляемых токенов.
    
    '''

    for _ in range(num_texts):
        print("_" * 50)
        tensor_prefix = torch.tensor([tokenizer.encode(prefix)]).to(device)
        generated_tokens = model.generate(
            tensor_prefix, max_length=max_length, do_sample=True, top_k=42, temperature=0.9, verbose=False
        )
        decoded_text = tokenizer.decode(generated_tokens[0])
        print(decoded_text)

In [11]:
# избавимся от избыточного логирования
logging.set_verbosity_error()

In [18]:
generate_texts(model, "Hi, I'm Vladimir", max_length=20)

__________________________________________________
Hi, I'm Vladimir. It's a great pleasure to welcome you. Welcome back to this show
__________________________________________________
Hi, I'm Vladimir Putin.

If there are any of you who've been following the
__________________________________________________
Hi, I'm Vladimir. I'm a developer, who helps people use their mobile devices in a
__________________________________________________
Hi, I'm Vladimir. I've been here since the beginning, but now I'm starting my
__________________________________________________
Hi, I'm Vladimir Ilyich Lenin! You're listening to The Lenin Show on Radio Free Europe


Видно, что довольно обдуманный текст, но давайте все же приступим к нашему заданию. Хорошо видно, что так как он обучался на большом количестве английской литературы, то он запомнил самых известных Владимиров (Путин и Ленин). Для этого будем использовать готовые функции для удобства. У трансформера есть свой реализованный [train](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Trainer.model). Это довольно сильно облегчит написание кода

Но на самом для нас самая большая проблема это создать датасет, все остальное должно делать довольно легко. Есть прекрасный класс TextDataset, вот [тут](https://github.com/huggingface/transformers/blob/main/src/transformers/data/datasets/language_modeling.py) больше информации есть. Еще одной идеей было использовать LineByLineTextDataset, но есть нюанс, что изначально gtp обучали без pad, то есть такого символа просто нет в токенизации, поэтому будет вылезать ошибка. Я нашел такое [обсуждение](https://github.com/huggingface/transformers/issues/4122) данной проблемы. В таком случае лучше использовать TextDataset, чем LineByLineTextDataset.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained("gpt2")

block_size=int(max_len*2)
train_dataset = TextDataset(tokenizer=tokenizer,file_path='./nietzsche.txt',block_size=block_size)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,
Token indices sequence length is longer than the specified maximum sequence length for this model (143770 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
model = AutoModelWithLMHead.from_pretrained("gpt2").to(device)
model.train(True)

training_args = TrainingArguments(
    output_dir="./history_model", 
    save_total_limit=2,
    num_train_epochs=50,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=8,
    disable_tqdm=False,
    save_steps=50,
    logging_steps=50,
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5), None)
)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:911: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [ ]:
def clean_gpu():
  global model,training_args,trainer
  del model,training_args,trainer
  torch.cuda.empty_cache()

In [ ]:
trainer.train()
torch.save(model, 'gpt_model_50_epochs')

***** Running training *****
  Num examples = 998
  Num Epochs = 50
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 8
  Total optimization steps = 350


Step,Training Loss
50,4.730600
100,4.454400
150,4.335800
200,4.262800
250,4.215000
300,4.186500
350,4.170800


Saving model checkpoint to ./history_model/checkpoint-50
Configuration saved in ./history_model/checkpoint-50/config.json
Model weights saved in ./history_model/checkpoint-50/pytorch_model.bin
Saving model checkpoint to ./history_model/checkpoint-100
Configuration saved in ./history_model/checkpoint-100/config.json
Model weights saved in ./history_model/checkpoint-100/pytorch_model.bin
Saving model checkpoint to ./history_model/checkpoint-150
Configuration saved in ./history_model/checkpoint-150/config.json
Model weights saved in ./history_model/checkpoint-150/pytorch_model.bin
Deleting older checkpoint [history_model/checkpoint-50] due to args.save_total_limit
Saving model checkpoint to ./history_model/checkpoint-200
Configuration saved in ./history_model/checkpoint-200/config.json
Model weights saved in ./history_model/checkpoint-200/pytorch_model.bin
Deleting older checkpoint [history_model/checkpoint-100] due to args.save_total_limit
Saving model checkpoint to ./history_model/check

In [ ]:
trainer.train()
torch.save(model, 'gpt_model_100_epochs')

***** Running training *****
  Num examples = 998
  Num Epochs = 50
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 8
  Total optimization steps = 350


Step,Training Loss
50,4.170700
100,4.170600


Saving model checkpoint to ./history_model/checkpoint-50
Configuration saved in ./history_model/checkpoint-50/config.json
Model weights saved in ./history_model/checkpoint-50/pytorch_model.bin
Deleting older checkpoint [history_model/checkpoint-300] due to args.save_total_limit
Saving model checkpoint to ./history_model/checkpoint-100
Configuration saved in ./history_model/checkpoint-100/config.json
Model weights saved in ./history_model/checkpoint-100/pytorch_model.bin
Deleting older checkpoint [history_model/checkpoint-350] due to args.save_total_limit


KeyboardInterrupt: ignored

Хорошо видно, что лосс улучшается уже почти незаметно, так что можно остановиться

In [ ]:
model.eval()
logging.set_verbosity_error()
generate_texts(model, 'people love ml and', max_length=50)

__________________________________________________
people love ml and their own kind. But the man who has never hitherto, at any rate, had the means to know when he was right
when he had been wrong, the man who has always had the means of being right and
never
__________________________________________________
people love ml and h to know how
they are not a burden to the world or their environment; the truth is, they are
mere means of obtaining the knowledge of an immense and
profoundly powerful world of which there is no small
__________________________________________________
people love ml and m. The only reason why people do such things is they hate the thing
they hate--it is only they who, after all, love and enjoy it. There is no place for
that; it is only they who
__________________________________________________
people love ml and pf men,
the "glamorous" man, the "sweet," and the "unconditional," may enjoy the same
special relationship with the feminine. In speaking of "love,"

In [ ]:
 generate_texts(model,  "artificial intelegence is interesting", max_length=50)

__________________________________________________
artificial intelegence is interesting and difficult to reconcile with what seems to be the fact of an "emotionally" connected state of the soul. The first step in the explanation of the fact of "emotions"--inasmuch
__________________________________________________
artificial intelegence is interesting, if
it may be that, in the minds of a great multitude of philosophers,
a great
intellect, a superintellect and a superdeterioration of the will and action,
__________________________________________________
artificial intelegence is interesting enough
even to the extent that it is generally perceived that the more
intelligent one is, the more difficult it is for the soul to deal with the "intelligences
of which it is so
__________________________________________________
artificial intelegence is interesting to consider, and
will not be allowed any more.


59

=How to Make Sense of a Man's World.=--I have hitherto treated of man's
world, 

In [ ]:
 generate_texts(model,  "be no mistake about it:", max_length=50)

__________________________________________________
be no mistake about it:
its existence, the nature of its subject, the "ultimate power"
of its object, its origin, which the reader has in mind, may be
a delusion, an error in judgment, such as a
__________________________________________________
be no mistake about it: the whole process
should be the responsibility of the individual, and should therefore be undertaken
according to the requirements set forth in the law.

234. In the event of a German revolt, the majority of Germans
__________________________________________________
be no mistake about it: the present has a new origin; the whole
origin of life has been acquired--an origin which is now far greater than before. What
has happened to the people of Europe since the beginning has
been the result
__________________________________________________
be no mistake about it:
It is the most delicate, unbreakable rule for all living beings
to maintain in mind that, whatever their own 

Получается связный текст, но нет ощущения, что это философский текст, так как наша модель обучалась очень много эпох и на больших других данных, а тут мы совсем немного поучили и на маленьких данных.

Давайте попробуем с другим lr, начнем с более большим

In [ ]:
model = AutoModelWithLMHead.from_pretrained("gpt2").to(device)
model.train(True)

training_args = TrainingArguments(
    output_dir="./history_model", 
    save_total_limit=2,
    num_train_epochs=50,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=8,
    disable_tqdm=False,
    save_steps=50,
    logging_steps=50,
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-4), None)
)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:911: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [ ]:
trainer.train()
torch.save(model, 'gpt_model_lr_-4_50_epochs_')

***** Running training *****
  Num examples = 998
  Num Epochs = 50
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 8
  Total optimization steps = 350


Step,Training Loss
50,4.243500
100,3.671700
150,3.283800
200,2.988000
250,2.751800
300,2.570900
350,2.472800


Saving model checkpoint to ./history_model/checkpoint-50
Configuration saved in ./history_model/checkpoint-50/config.json
Model weights saved in ./history_model/checkpoint-50/pytorch_model.bin
Saving model checkpoint to ./history_model/checkpoint-100
Configuration saved in ./history_model/checkpoint-100/config.json
Model weights saved in ./history_model/checkpoint-100/pytorch_model.bin
Saving model checkpoint to ./history_model/checkpoint-150
Configuration saved in ./history_model/checkpoint-150/config.json
Model weights saved in ./history_model/checkpoint-150/pytorch_model.bin
Deleting older checkpoint [history_model/checkpoint-50] due to args.save_total_limit
Saving model checkpoint to ./history_model/checkpoint-200
Configuration saved in ./history_model/checkpoint-200/config.json
Model weights saved in ./history_model/checkpoint-200/pytorch_model.bin
Deleting older checkpoint [history_model/checkpoint-100] due to args.save_total_limit
Saving model checkpoint to ./history_model/check

In [ ]:
trainer.train()
torch.save(model, 'gpt_model_lr_-4_epochs_100')

***** Running training *****
  Num examples = 998
  Num Epochs = 50
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 8
  Total optimization steps = 350


Step,Training Loss
50,2.454600
100,2.456800
150,2.456200
200,2.457000
250,2.454300
300,2.458000


Saving model checkpoint to ./history_model/checkpoint-50
Configuration saved in ./history_model/checkpoint-50/config.json
Model weights saved in ./history_model/checkpoint-50/pytorch_model.bin
Deleting older checkpoint [history_model/checkpoint-300] due to args.save_total_limit
Saving model checkpoint to ./history_model/checkpoint-100
Configuration saved in ./history_model/checkpoint-100/config.json
Model weights saved in ./history_model/checkpoint-100/pytorch_model.bin
Deleting older checkpoint [history_model/checkpoint-350] due to args.save_total_limit
Saving model checkpoint to ./history_model/checkpoint-150
Configuration saved in ./history_model/checkpoint-150/config.json
Model weights saved in ./history_model/checkpoint-150/pytorch_model.bin
Deleting older checkpoint [history_model/checkpoint-50] due to args.save_total_limit
Saving model checkpoint to ./history_model/checkpoint-200
Configuration saved in ./history_model/checkpoint-200/config.json
Model weights saved in ./history_m

Давайте попробуем уменьшить lr

In [ ]:
model = torch.load('gpt_model_lr_-4_epochs_100')
model.train(True)

training_args = TrainingArguments(
    output_dir="./history_model", 
    save_total_limit=2,
    num_train_epochs=50,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=8,
    disable_tqdm=False,
    save_steps=50,
    logging_steps=50,
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5), None)
)

In [ ]:
trainer.train()
torch.save(model, 'gpt_model_lr_-4_to_-5_epochs_150')

***** Running training *****
  Num examples = 998
  Num Epochs = 50
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 8
  Total optimization steps = 350


Step,Training Loss
50,2.400300
100,2.333200
150,2.288900
200,2.263600
250,2.246100
300,2.257600
350,2.282400


Saving model checkpoint to ./history_model/checkpoint-50
Configuration saved in ./history_model/checkpoint-50/config.json
Model weights saved in ./history_model/checkpoint-50/pytorch_model.bin
Saving model checkpoint to ./history_model/checkpoint-100
Configuration saved in ./history_model/checkpoint-100/config.json
Model weights saved in ./history_model/checkpoint-100/pytorch_model.bin
Saving model checkpoint to ./history_model/checkpoint-150
Configuration saved in ./history_model/checkpoint-150/config.json
Model weights saved in ./history_model/checkpoint-150/pytorch_model.bin
Deleting older checkpoint [history_model/checkpoint-50] due to args.save_total_limit
Saving model checkpoint to ./history_model/checkpoint-200
Configuration saved in ./history_model/checkpoint-200/config.json
Model weights saved in ./history_model/checkpoint-200/pytorch_model.bin
Deleting older checkpoint [history_model/checkpoint-100] due to args.save_total_limit
Saving model checkpoint to ./history_model/check

In [ ]:
model.eval()
logging.set_verbosity_error()
generate_texts(model, 'people love ml and', max_length=50)
print("_" * 50)
generate_texts(model,  "artificial intelegence is interesting", max_length=50)
print("_" * 50)
generate_texts(model,  "be no mistake about it:", max_length=50)

__________________________________________________
people love ml and no
more, for they are afraid of losing their taste. "Everything is too
imperfect"--they always tell themselves after many pokes and lacerations, at
which, indeed, many a mediocre man will
__________________________________________________
people love ml and know how to
betray themselves--because in all honesty, it turns out that in _everything_
the love of a thing_ it is _absolutely_ NOT the love of a God
and _absolutely NOT the love
__________________________________________________
people love ml and men,"
--they must exchange for each other the worst instincts of
aggression and hate, hatred and modesty.

271. In a certain period of the Russian Empire Christians really
did not believe as they did that
__________________________________________________
people love ml and hate it! But ye must learn to love it also, and for ever
more
love it!"

212. "It gives me joy, it makes me sad, it puts all my worries into
order,

Текст стал более философским, причем некоторые фразы очень философские, особенно "be no mistake about it: the more
dangerous philosophers nowadays are, the more likely they are to make mistakes in
the long run;", "не заблуждайтесь на этот счет: он именно
обезьяна - обезьяна, которая НЕ МОЖЕТ научиться быть нравственной; на самом деле, он, возможно, обезьяна, которая НЕ МОЖЕТ научиться быть хорошей".

Теперь давайте попробуем с lr=1e-3 с scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

In [ ]:
model = AutoModelWithLMHead.from_pretrained("gpt2").to(device)
model.train(True)

training_args = TrainingArguments(
    output_dir="./history_model", 
    save_total_limit=2,
    num_train_epochs=50,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=8,
    disable_tqdm=False,
    save_steps=50,
    logging_steps=50,
    )

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (optimizer, torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=16*8))
)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:911: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [ ]:
trainer.train()
torch.save(model, 'gpt_model_CosineAnnealingLR_lr=1e-3_epochs_50')

***** Running training *****
  Num examples = 998
  Num Epochs = 50
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 8
  Total optimization steps = 350


Step,Training Loss
50,3.765900
100,1.324400
150,0.716400
200,0.538800
250,0.273800
300,0.143000
350,0.063200


Saving model checkpoint to ./history_model/checkpoint-50
Configuration saved in ./history_model/checkpoint-50/config.json
Model weights saved in ./history_model/checkpoint-50/pytorch_model.bin
Saving model checkpoint to ./history_model/checkpoint-100
Configuration saved in ./history_model/checkpoint-100/config.json
Model weights saved in ./history_model/checkpoint-100/pytorch_model.bin
Saving model checkpoint to ./history_model/checkpoint-150
Configuration saved in ./history_model/checkpoint-150/config.json
Model weights saved in ./history_model/checkpoint-150/pytorch_model.bin
Deleting older checkpoint [history_model/checkpoint-50] due to args.save_total_limit
Saving model checkpoint to ./history_model/checkpoint-200
Configuration saved in ./history_model/checkpoint-200/config.json
Model weights saved in ./history_model/checkpoint-200/pytorch_model.bin
Deleting older checkpoint [history_model/checkpoint-100] due to args.save_total_limit
Saving model checkpoint to ./history_model/check

Как видим мы получили потрясающий результат метрики намного лучше, но давайте попробуем еще дообучить

In [ ]:
trainer.train()
torch.save(model, 'gpt_model_CosineAnnealingLR_lr=1e-3_epochs_100')

***** Running training *****
  Num examples = 998
  Num Epochs = 50
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 8
  Total optimization steps = 350


Step,Training Loss
50,0.044900
100,0.041800
150,0.043500
200,0.051700
250,0.042200
300,0.030300
350,0.029100


Saving model checkpoint to ./history_model/checkpoint-50
Configuration saved in ./history_model/checkpoint-50/config.json
Model weights saved in ./history_model/checkpoint-50/pytorch_model.bin
Deleting older checkpoint [history_model/checkpoint-300] due to args.save_total_limit
Saving model checkpoint to ./history_model/checkpoint-100
Configuration saved in ./history_model/checkpoint-100/config.json
Model weights saved in ./history_model/checkpoint-100/pytorch_model.bin
Deleting older checkpoint [history_model/checkpoint-350] due to args.save_total_limit
Saving model checkpoint to ./history_model/checkpoint-150
Configuration saved in ./history_model/checkpoint-150/config.json
Model weights saved in ./history_model/checkpoint-150/pytorch_model.bin
Deleting older checkpoint [history_model/checkpoint-50] due to args.save_total_limit
Saving model checkpoint to ./history_model/checkpoint-200
Configuration saved in ./history_model/checkpoint-200/config.json
Model weights saved in ./history_m

Как хорошо видно, метрика уже не так сильно улучшается, давайте протестируем на генерации текста

In [ ]:
model.eval()
logging.set_verbosity_error()
generate_texts(model, 'people love ml and', max_length=50)
print("_" * 50)
generate_texts(model,  "artificial intelegence is interesting", max_length=50)
print("_" * 50)
generate_texts(model,  "be no mistake about it:", max_length=50)

__________________________________________________
people love ml and play; I rather think that as the love
of mankind grows, the more firmly and forever chained to its place
in the heart of mankind, the more polyphone the music and the sounding of its
effects, the more
__________________________________________________
people love ml and play; it is strange to them if they do not
 love one another!

51. In praise there is more obtrusiveness than in blame.

52. What a person IS begins to betray itself when
__________________________________________________
people love ml and plays THIS bad
game.

206. In relation to the genius, that is to say, a being who either
ENGENDERS or PRODUCES--both words understood in their fullest sense--the
man
__________________________________________________
people love ml and play; I even
themselves do not know what they are, and what they serve us
as a good conscience. But, as has been pointed out,
this is a realm of questions and answers in which a more

Получили более философский текст, но есть местами неидеально связный текст, скорее всего, это связано с тем, что мы начали сильно переобучаться под определенный текст, но давайте посмотрим, что покажет данная модель на следующем примере.

In [ ]:
generate_texts(model, "Hi, I'm Vladimir", max_length=20)

__________________________________________________
Hi, I'm Vladimir--I hope you understand
that I am not a skeptic. I
__________________________________________________
Hi, I'm Vladimir--I know you well enough, my friends. It
is the appearance
__________________________________________________
Hi, I'm Vladimir--I know that already.

220. I may be mistaken about
__________________________________________________
Hi, I'm Vladimir--I know you well enough how to help me!
with a good
__________________________________________________
Hi, I'm Vladimir, and I know that
there is a second, also, by means


Hi, I'm Vladimir--I hope you understand
that I am not a skeptic. Оч смешная и философская цитата получилась, очень показательно если сравнить с генерацией в начале задания только на предобученной модели, то он учитывал мировых политических лидеров (известных людей с именем Владимир). Теперь же он генерирует философский текст, для человека с именем Владимир.

Давайте попробуем теперть повторить предыдущий пункт, но с более маленьким lr.

In [ ]:
model = AutoModelWithLMHead.from_pretrained("gpt2").to(device)
model.train(True)

training_args = TrainingArguments(
    output_dir="./history_model", 
    save_total_limit=2,
    num_train_epochs=100,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=8,
    disable_tqdm=False,
    save_steps=50,
    logging_steps=50,
    )

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (optimizer, torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=16*8))
)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:911: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [ ]:
trainer.train()
torch.save(model, 'gpt_model_CosineAnnealingLR_lr=1e-4_epochs_100')

***** Running training *****
  Num examples = 998
  Num Epochs = 100
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 8
  Total optimization steps = 700


Step,Training Loss
50,4.244900
100,3.755500
150,3.652800
200,3.592600
250,3.337500
300,2.962900
350,2.649600
400,2.546900
450,2.453200
500,2.222200


Saving model checkpoint to ./history_model/checkpoint-50
Configuration saved in ./history_model/checkpoint-50/config.json
Model weights saved in ./history_model/checkpoint-50/pytorch_model.bin
Deleting older checkpoint [history_model/checkpoint-300] due to args.save_total_limit
Saving model checkpoint to ./history_model/checkpoint-100
Configuration saved in ./history_model/checkpoint-100/config.json
Model weights saved in ./history_model/checkpoint-100/pytorch_model.bin
Deleting older checkpoint [history_model/checkpoint-350] due to args.save_total_limit
Saving model checkpoint to ./history_model/checkpoint-150
Configuration saved in ./history_model/checkpoint-150/config.json
Model weights saved in ./history_model/checkpoint-150/pytorch_model.bin
Deleting older checkpoint [history_model/checkpoint-50] due to args.save_total_limit
Saving model checkpoint to ./history_model/checkpoint-200
Configuration saved in ./history_model/checkpoint-200/config.json
Model weights saved in ./history_m

Мы получили довольно хороший лосс по итогу, хотя он намного больше, чем когда мы пробовали с lr=1e-3

In [ ]:
model.eval()
logging.set_verbosity_error()
generate_texts(model, 'people love ml and', max_length=50)
print("_" * 50)
generate_texts(model,  "artificial intelegence is interesting", max_length=50)
print("_" * 50)
generate_texts(model,  "be no mistake about it:", max_length=50)

__________________________________________________
people love ml and romania more than we love the
martyr,"--this love has an exceedingly rapid growth, for the very reason
that on the one hand the plebeian and plebeian eye has long ago
looked
__________________________________________________
people love ml and m
in me! What! A man of science who has failed to see that science
himself is a MIRROR--PLATONISM! Oh, gosh! Such an INDIFFERENCE of
things
__________________________________________________
people love ml and loon.

201. What!? A philosopher? Nay, only a poet? But you misunderstand me--when
YOU attach so much importance to a philosopher, you are only addressing
a personal attack on him--name
__________________________________________________
people love ml and nothing else. There may even be
more in them, hidden under the skin, mingled with fishnet threadbare feet and
hair, like fine beads of silver. But this is neither here, nor
there anything else
___________________________

In [ ]:
generate_texts(model, "Hi, I'm Vladimir", max_length=20)

__________________________________________________
Hi, I'm Vladimir Putin.)

236. The Russian Empire as a Continent?


__________________________________________________
Hi, I'm Vladimir Putin!"

189. He who attains his ideal, gets along
__________________________________________________
Hi, I'm Vladimir Nabokov, and now I'm playing the
secretary of state
__________________________________________________
Hi, I'm Vladimir Nabokov!" And thus
the great, all powerful, all historical
__________________________________________________
Hi, I'm Vladimir Putin," he says,
on the contrary of all the vivis


Тут очень показательно, что с именем Владимир он все еще предпочитает генерировать текст из статей, на которых он обучался (то есть он упоминает известных людей с именем Владимир)

**Вывод:** Сначала скажу про токенизацию. Как говорилось раньше нам необходима BPE, так как это более оптимально для связной и хорошей генерации текста. Но в gpt2 использует даже более хитрую вещь, а именно BPE на уровне байтов. Также я перебрал различные lr и с/без schedule. Лучшие результаты показал lr=1e-3 с schedule. Именно он дал результат на моей цитате не похожий на результат "голой" модели, то есть он отдает предпочтению текстам ницше, а не предобученной моделью. Все остальные модели цитату "Hi, I'm Vladimir" продолжают используя фамилии известных людей, а не начинают философски излагать мысль. Но при этом возможно что самая лучшая модель начинает переобучаться под данный текст, и немного терять логичность и связность в высказывании. Также хотелось бы проговорить почему было неактуально с 0 обучать нашу модель. Для обучения трансформеров используют очень большое кол-во данных, то есть для gpt2 использовали порядка 40 gb. Так что было нецелесообразно обучать с 0 нашу модель на наших маленьких даннных, скорее всего мы бы даже не уловили качественную связь в наших данных.

Подумайте внимательно о токенизации, параметрах обучения